В этом блокноте мы создаем синтетические данные для обучения GLiNER. 
План:
1) Мы берем случайную статью из википедии. 
2) Просим Большую Языковую Модель выписать все сущности из текста в определнном формате
3) Парсим выход

In [2]:
import json
# У мистраля бесплатное API для LLM, работает без VPN, но можно переписать на другое при желании
from mistralai import Mistral
import traceback # в случае ошибки, при использовании try-except, выводит полный лог
import time

# Сохранять предсказания из dict, в json
def save_data_to_file(data, filepath):
    """Saves the processed data to a JSON file."""
    with open(filepath, 'w') as f:
        json.dump(data, f)

# нужно вставить свой ключ
api_key = "mXhjdibfW9ay0Crp7LQ6ZHGlP9cTVFnN" # дарю

# выбрать модель 
model = "mistral-large-latest"
# model = "open-mistral-nemo"

client = Mistral(api_key=api_key)

def generate(prompt):

    chat_response = client.chat.complete(
        model= model,
        messages = [
            {
                "role": "user",
                "content": prompt,
            },
        ]
    )
    return chat_response.choices[0].message.content
    # outputs = llm.generate([create_json_prompt_for_synthetic_data(**kwargs)], sampling_params)
    # return json.loads(outputs[0].outputs[0].text)
    

In [3]:
import pandas as pd

addr = pd.read_csv("../volgait2024-semifinal-addresses.csv", sep=';')
df = pd.read_csv("../volgait2024-semifinal-task.csv", sep=';')

In [4]:
def create_json_prompt_for_text_analysis(text):
  prompt = """
**Objective:**
Analyze the given text and extract addresses. Each address should include the street and all the buildings on it. Ensure that each address is listed separately, even if multiple addresses are mentioned in a single sentence. Pay attention to specific notations such as "ч/д" which indicate only even-numbered buildings. The text should be quoted exactly as it appears, without changing cases, abbreviations, etc.

**Format Requirements:**
- The output should be formatted in JSON, containing the text and the corresponding entities list.
- Each address in the text should be accurately marked and annotated in the 'entities' list.
- Meticulously follow all the listed attributes.
- The text should be quoted exactly as it appears, without changing cases, abbreviations, etc.
- Ensure that the extracted addresses match the original text exactly, without adding or removing any parts.
- Do not explain your answer. Just output > ```json\n ...text... \n```

**Entity Annotation Details:**
- All entity types must be in lowercase. For example, use "address" not "ADDRESS".
- Entity types can be multiwords separate by space. For instance, use "street name" rather than "street_name".
- Entities spans can be nested within other entities.
- A single entity may be associated with multiple types. list them in the key "types".

**Output Schema:**


```json
{
  "text": "{text}",
  "entities": [
    {"entity": "street name", "types": ["street"]},
    ...
  ]
}
```

Example:

Input text was:
"Ульяновск г, Нефтяников проезд, д. 6, 4 замена ввода Д=90мм.без воды ул Аблукова 43 замена ПГ. без воды Тельмана 24.26.28.30"

Correct output:

```json
{
  "text": "из п\з д200  Без хвс Нариманова 1,1г,3,5, ч/д в границах: Кооперат.- пер.Новикова- Лермонтова- Нариманова",
  "entities": [
    {"entity": "Нариманова 1,1г,3,5", "types": ["Полный адрес"]},
    {"entity": "Нариманова ч/д", "types": ["Полный адрес"]},
    {"entity": "Кооперат.- пер.Новикова- Лермонтова- Нариманова", "types": ["Полный адрес"]}
  ]
}
```

Example 2:

Input text was:
"Р. Люксембург 3 утечка из-под земли д=150, без ХВС Р. Люксембург 1-5, Бакинская 22-28, 27-35."

Incorrect output:

```json
{
  "text": "Р. Люксембург 3 утечка из-под земли д=150, без ХВС Р. Люксембург 1-5, Бакинская 22-28, 27-35.",
  "entities": [
    {"entity": "Р. Люксембург 3", "types": ["Полный адрес"]},
    {"entity": "Р. Люксембург 1-5", "types": ["Полный адрес"]},
    {"entity": "Бакинская 22-28", "types": ["Полный адрес"]},
    {"entity": "Бакинская 27-35", "types": ["Полный адрес"]}
  ]
}

That output was incorrect, because "Бакинская 27-35" is invalid substring. You are only allowed to extract substrings, the same way they are formatted in the text. 

Correct output:

```json
{
  "text": "Р. Люксембург 3 утечка из-под земли д=150, без ХВС Р. Люксембург 1-5, Бакинская 22-28, 27-35.",
  "entities": [
    {"entity": "Р. Люксембург 3", "types": ["Полный адрес"]},
    {"entity": "Р. Люксембург 1-5", "types": ["Полный адрес"]},
    {"entity": "Бакинская 22-28, 27-35", "types": ["Полный адрес"]}
  ]
}
```

Example output 3:

```json
{
  "text": "Луговое с, Пензенская ул, д. 7, 6, 8, 3, 9, 1, 11, 2, 4, 10, 5 Луговое с, Пензенский пер, д. 2, 1, 7, 5, 3, 9, 4 Луговое с, Дорожная ул, д. 5, 1, 7, 3, 0 Луговое с, Молодежная ул, д. 3, 8, 2, 7, 5, 6, 9, 10, 12 Луговое с, Полевая ул, д. 1 Луговое с, Школьная ул, д. 20 Луговое с, Первомайская ул, д. 23, 21, 29",
  "entities": [
    {"entity": "Пензенская ул, д. 7, 6, 8, 3, 9, 1, 11, 2, 4, 10, 5", "types": ["Полный адрес"]},
    {"entity": "Пензенский пер, д. 2, 1, 7, 5, 3, 9, 4", "types": ["Полный адрес"]},
    {"entity": "Дорожная ул, д. 5, 1, 7, 3, 0", "types": ["Полный адрес"]},
    {"entity": "Молодежная ул, д. 3, 8, 2, 7, 5, 6, 9, 10, 12", "types": ["Полный адрес"]},
    {"entity": "Полевая ул, д. 1", "types": ["Полный адрес"]},
    {"entity": "Школьная ул, д. 20", "types": ["Полный адрес"]},
    {"entity": "Первомайская ул, д. 23, 21, 29", "types": ["Полный адрес"]}
  ]
}
```

  """

  # Adding the dynamically created text to the prompt
  prompt += f"""

  Text:
  "{text}"
  """

  return prompt

#Пример использования функции
text = "Ульяновск г, Светлый 3-й пер, д. 3, 7, 1, 11, 5, 13, 9 Ульяновск г, Светлая ул, д. 1, 7, 9, 3, 5, 11, 2, 6"
print(create_json_prompt_for_text_analysis(text))


**Objective:**
Analyze the given text and extract addresses. Each address should include the street and all the buildings on it. Ensure that each address is listed separately, even if multiple addresses are mentioned in a single sentence. Pay attention to specific notations such as "ч/д" which indicate only even-numbered buildings. The text should be quoted exactly as it appears, without changing cases, abbreviations, etc.

**Format Requirements:**
- The output should be formatted in JSON, containing the text and the corresponding entities list.
- Each address in the text should be accurately marked and annotated in the 'entities' list.
- Meticulously follow all the listed attributes.
- The text should be quoted exactly as it appears, without changing cases, abbreviations, etc.
- Ensure that the extracted addresses match the original text exactly, without adding or removing any parts.
- Do not explain your answer. Just output > ```json
 ...text... 
```

**Entity Annotation Details:**


## Функции, которые распарсят ответ модели, и переведут его в формат, который подходит для обучения модели. 

Это переделанный код из примеров с гитхаба. Тут не очень красиво. 

Эти функции переформатируют ответы LLM, в формат, который поддерживает GLiNER для обучения. 

In [5]:
import re

# Эта функция делит на слова
def tokenize_text(text):
    """Tokenize the input text into a list of tokens."""
    text = text.replace('-', ' ')

    return re.findall(r'\w+(?:[-_]\w+)*|\S', text)

# Находит срез (номера слов начала и конца), для слов которые мы нашли
# По сути тут просто алгоритм поиска, но написанный на коленке, для работы с массивами
def extract_entities(data):
    all_examples = []

    for dt in data:

        # Attempt to extract entities; skip current record on failure
        try:
            tokens = tokenize_text(dt['text'])
            ents = [(k["entity"], k["types"]) for k in dt['entities']]
        except:
            continue

        spans = []
        for entity in ents:
            entity_tokens = tokenize_text(str(entity[0]))

            # Find the start and end indices of each entity in the tokenized text
            for i in range(len(tokens) - len(entity_tokens) + 1):
                if " ".join(tokens[i:i + len(entity_tokens)]).lower() == " ".join(entity_tokens).lower():
                    for el in entity[1]:
                        spans.append((i, i + len(entity_tokens) - 1, el.lower().replace('_', ' ')))

        # Append the tokenized text and its corresponding named entity recognition data
        all_examples.append({"tokenized_text": tokens, "ner": spans})

    return all_examples

# generation functions
def generate_from_prompts(prompts, llm, sampling_params):
    outputs = llm.generate(prompts, sampling_params)

    all_outs = []
    
    for output in outputs:
        try:
            js = json.loads(output.outputs[0].text.strip())
        except:
            continue
            
        all_outs.append(js)

    return all_outs, extract_entities(all_outs)

### Generate outputs

In [8]:
processed_output = []


output_file = "gliner-jkh-clean.json"

# with open(output_file, "r") as f:
#     processed_output = json.load(f)

print('Dataset size:', len(processed_output))


for i in df['comment'][len(processed_output):]:
    text = i

    prompt = create_json_prompt_for_text_analysis(text)

    answer = generate(prompt)
    print(answer)

    answer = answer.replace('```json', '').replace('```', '')

    try:
        js = json.loads(answer.strip())
    except:
        print("Error while parsing json!!!")
        print("Here is that faulty json:")
        print(answer)
        continue
        # break


    # Attempt to extract entities; skip current record on failure
    try:
        tokens = tokenize_text(js['text'])
        ents = [(k["entity"], k["types"]) for k in js['entities']]
    except:
        print("Failed at entities step!")
        traceback.print_exc()
        continue
        # break

    spans = []
    for entity in ents:
        entity_tokens = tokenize_text(str(entity[0]))

        # Find the start and end indices of each entity in the tokenized text
        for i in range(len(tokens) - len(entity_tokens) + 1):
            if " ".join(tokens[i:i + len(entity_tokens)]).lower() == " ".join(entity_tokens).lower():
                for el in entity[1]:
                    spans.append((i, i + len(entity_tokens) - 1, el.lower().replace('_', ' ')))

        # Append the tokenized text and its corresponding named entity recognition data
    print({"tokenized_text": tokens, "ner": spans})
    print( len(processed_output), '----', '\n\n')

    processed_output.append({"tokenized_text": tokens, "ner": spans})

    # break
    save_data_to_file(processed_output, output_file)

    time.sleep(10)
    

Dataset size: 0
```json
{
  "text": "Новосельдинская 7 утечка из-под земли д=100, без ХВС пос. Новосельдинский.",
  "entities": [
    {"entity": "Новосельдинская 7", "types": ["Полный адрес"]}
  ]
}
```
{'tokenized_text': ['Новосельдинская', '7', 'утечка', 'из', 'под', 'земли', 'д', '=', '100', ',', 'без', 'ХВС', 'пос', '.', 'Новосельдинский', '.'], 'ner': [(0, 1, 'полный адрес')]}
0 ---- 


```json
{
  "text": "из п/з Д=400мм ул,Шолмова 9",
  "entities": [
    {"entity": "ул,Шолмова 9", "types": ["Полный адрес"]}
  ]
}
```
{'tokenized_text': ['из', 'п', '/', 'з', 'Д', '=', '400мм', 'ул', ',', 'Шолмова', '9'], 'ner': [(7, 10, 'полный адрес')]}
1 ---- 


```json
{
  "text": "замена труб в подъезде ж/д отключение ГВС и ХВС",
  "entities": []
}
```
{'tokenized_text': ['замена', 'труб', 'в', 'подъезде', 'ж', '/', 'д', 'отключение', 'ГВС', 'и', 'ХВС'], 'ner': []}
2 ---- 


```json
{
  "text": "п. Новосельдинский 7- утечка из кол-ца Д 100, без ХВС п. Новосельдинский -100 ч/д",
  "entities": 

SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}